In [ ]:
%%capture
%load_ext rich
%pip install pandas matplotlib plotly 'nbformat>=4.2.0' 'ipywidgets>=7.6'
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from quisp import (
    QNode,
    Network,
    ChannelOption,
    QNodeAddr,
    NativeSimulator,
    Config,
    NetworkBuilder,
    LinkType,
)

from analysis_utils import count_rulesets_by_responders

def generate_network(num_nodes: "int", num_subnets: "int") ->"Network":
    assert num_nodes >= 2
    assert num_subnets >= 0
    subnet_size = num_nodes // num_subnets if num_subnets != 0 else 0

    n = NetworkBuilder(f"linear_{num_nodes}nodes_{num_subnets}subnets_internetworking")
    network_addr = 1
    subnet_addr = num_subnets * 2
    for i in range(1, num_nodes+1):
        addr_list = [QNodeAddr(network_addr, i)]
        if subnet_size > 0:
            if i % subnet_size == 1: # new subnet starts
                network_addr += 1
                subnet_addr += 1
                addr_list.append(QNodeAddr(subnet_addr, i))
            elif i % subnet_size == 0: # end of subnets
                network_addr += 1
                addr_list.append(QNodeAddr(subnet_addr, i))
            else: # in subnet, no top level addr
                addr_list = [QNodeAddr(subnet_addr, i)]
        
        n.add_qnode(addr_list, is_initiator=(i == 1), is_resipient=(i == num_nodes))
    n.connect_linear(ChannelOption(quantum_channel_distance=5, classical_channel_distance=5))
    return n.network





In [ ]:
plan = dict()

for num_nodes in [5,10, 15, 20, 25, 30, 35, 40, 45, 50]:
    for num_subnets in [0, 1,2,3,4,5]: #range(0, num_nodes // 3, 4):
        if num_subnets * 3 > num_nodes:
            continue
        network = generate_network(num_nodes=num_nodes, num_subnets=num_subnets)
        key = network.name
        plan[key] = {
            "network_name": key,
            "network": network,
            "num_nodes": num_nodes,
            "num_subnets": num_subnets,
            "error": None
        }
        
for k in plan:
    print(k)

print(len(plan), "networks")

In [ ]:
for key in plan:
    sim = NativeSimulator("../quisp")
    try:
        sim.load(plan[key]["network"])
        plan[key]["ned_dump"]= sim.network.dump()
        plan[key]["ini_dump"]= sim.config.dump()
        await sim.run()
    except RuntimeError:
        plan[key]["error"] = sim.error_messages
    sim.read_result()
    plan[key]["df"] = sim.df.copy()
    plan[key]["num_rulesets_df"] = count_rulesets_by_responders(sim.df)
    print(count_rulesets_by_responders(sim.df))

In [ ]:
from rich import progress
from rich.console import Console, Group
from rich.progress import Progress
from rich.live import Live


def run_with_console():
    console = Console()
    # Create a progress bar with the total number of events to simulate
    total_progress = Progress(
        "[status]{task.description}",
        progress.BarColumn(),
        "[progress.percentage]{task.percentage:>3.0f}%",
        "[progress.bar]{task.completed}/{task.total}",
        progress.TimeElapsedColumn(),
    )
    # Create a progress bar for each of the simulations
    sim_progress =  Progress(
        progress.SpinnerColumn(),
        "[task_name]{task.fields[sim_name]}",
        "[status]{task.description}",
        "[num_events]{task.fields[num_events]} events",
        "[ev_per_sec]{task.fields[ev_per_sec]} ev/sec",
        progress.TimeElapsedColumn(),
        console=console,
    )
    progress_group = Group(Group(sim_progress), total_progress)  # type: ignore

    with Live(progress_group, console=console) as live:
        live.console.log("[status]Starting Simulation")
        # Create the total progress task
        total_progress_task = total_progress.add_task("total", total=10)
run_with_console()


In [ ]:
plan = dict()

for num_nodes in [5,10, 15, 20, 25, 30, 35, 40, 45, 50]:
    for num_subnets in [0, 1,2,3,4,5]: #range(0, num_nodes // 3, 4):
        if num_subnets * 3 > num_nodes:
            continue
        network = generate_network(num_nodes=num_nodes, num_subnets=num_subnets)
        key = network.name
        plan[key] = {
            "network_name": key,
            "network": network,
            "num_nodes": num_nodes,
            "num_subnets": num_subnets,
            "error": None
        }
        
for k in plan:
    print(k)

print(len(plan), "networks")

# run simulations with generated plan dict. the dict contains each simulation setting.
async def run_all(plan: "Dict"):
    simulation_queue = [plan[key] for key in plan]
    
